# Bedienungsanleitung: Deine Finanzplanung in zwei Schritten
Dieses Tool hilft dir dabei, die Auswirkungen von Inflation und Zinseszins auf deine Altersvorsorge zu verstehen. Folge diesen zwei Schritten für eine realistische Prognose:

# 1. Schritt: Bedarfsanalyse (Kaufkraft & Rentenlücke)
Bevor du investierst, musst du wissen, wie viel Geld du im Alter tatsächlich benötigst. Die Inflation sorgt dafür, dass Preise steigen – 2.000 € haben in 40 Jahren eine deutlich geringere Kaufkraft als heute.

Ziel: Ermittle deine Rentenlücke.

Vorgehen: Gib deine heutigen monatlichen Kosten und deine erwartete gesetzliche Rente ein.

Ergebnis: Die Grafik zeigt dir im gelben Bereich die monatliche Summe an, die dir im Alter fehlen wird. Merke dir diesen Wert für Schritt 2.

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact

def kaufkraft_prognose_luecke(
    netto_monatlich,
    aktuelles_alter,
    rentenalter,
    inflation_prozent,
    erwartete_rente
):
    inflationsrate = inflation_prozent / 100
    lebenserwartung = 89

    alter = np.arange(aktuelles_alter, lebenserwartung + 1)
    jahre_differenz = alter - aktuelles_alter

    # Berechnung des nominalen Bedarfs (Kostensteigerung)
    bedarf_nominal = netto_monatlich * ((1 + inflationsrate) ** jahre_differenz)

    # Indizes und Werte für Renteneintritt
    idx_rente = max(0, rentenalter - aktuelles_alter)
    bedarf_bei_rente = bedarf_nominal[idx_rente]
    
    # Rentenlücke berechnen (nur ab Rentenalter relevant)
    luecke = max(0, bedarf_bei_rente - erwartete_rente)

    # Plotting
    plt.figure(figsize=(12, 6))
    
    # 1. Zone: Inflation (Bedarfsanstieg)
    plt.fill_between(alter, netto_monatlich, bedarf_nominal, color='#e74c3c', alpha=0.1, label="Inflations-Bedarf")
    plt.plot(alter, bedarf_nominal, color='#c0392b', linewidth=3, label="Zukünftige Lebenshaltungskosten")
    
    # 2. Zone: Rentenlücke (ab Renteneintritt)
    if erwartete_rente > 0:
        # Horizontale Linie für die Rente ab Rentenalter
        alter_rente = alter[idx_rente:]
        plt.hlines(erwartete_rente, rentenalter, lebenserwartung, colors='#f1c40f', linestyles='-', linewidth=3, label="Erwartete Rente")
        
        # Markierung der Lücke
        plt.fill_between(alter_rente, erwartete_rente, bedarf_nominal[idx_rente:], 
                         color='#f1c40f', alpha=0.4, label=f"Rentenlücke ({luecke:,.0f} €)")
        
        # Annotation der Lücke
        plt.annotate(f'Lücke: {luecke:,.0f} €/Monat', 
                     (rentenalter + (lebenserwartung-rentenalter)/2, (erwartete_rente + bedarf_bei_rente)/2),
                     ha='center', fontweight='bold', bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.8))

    # Markierungen und Design
    plt.axhline(netto_monatlich, color='#7f8c8d', linestyle='--', alpha=0.5, label="Heutige Kaufkraft")
    plt.axvline(rentenalter, color='#2c3e50', linestyle=':', alpha=0.7)
    
    plt.title(f"Analyse: Kaufkraft & Rentenlücke\n"
              f"Bedarf mit {rentenalter} J.: {bedarf_bei_rente:,.0f} € | "
              f"Heutiger Wert: {netto_monatlich:,.0f} €", fontsize=12)
    
    plt.ylabel("Monatlicher Betrag (€)")
    plt.xlabel("Alter (Jahre)")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend(loc='upper left', frameon=True)
    
    # Währung formatieren
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',').replace(',', '.') + ' €'))
    
    plt.tight_layout()
    plt.show()

# Widget-Konfiguration
ui_style = {'description_width': '220px'}
ui_layout = {'width': '550px'}

interact(
    kaufkraft_prognose_luecke,
    netto_monatlich=widgets.IntSlider(value=2500, min=500, max=10000, step=100, description="Heutige Kosten (€)", style=ui_style, layout=ui_layout),
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, step=1, description="Aktuelles Alter", style=ui_style, layout=ui_layout),
    rentenalter=widgets.IntSlider(value=67, min=25, max=75, step=1, description="Renteneintritt", style=ui_style, layout=ui_layout),
    inflation_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Kalk. Inflation (%)", style=ui_style, layout=ui_layout),
    erwartete_rente=widgets.IntSlider(value=0, min=0, max=10000, step=50, description="Erwartete Rente (€)", style=ui_style, layout=ui_layout)
);

interactive(children=(IntSlider(value=2500, description='Heutige Kosten (€)', layout=Layout(width='550px'), ma…

# 2. Schritt: Investment-Strategie (ETF-Rechner)
Hier berechnest du, wie du die zuvor ermittelte Lücke durch einen ETF-Sparplan schließen kannst.

Ziel: Prüfe, ob dein Kapital bis zum Lebensende reicht.

Vorgehen: 
1. Trage bei "Wunsch-Entnahme" den Wert der Rentenlücke aus Schritt 1 ein. 
2. Passe deine "Sparrate" und die "Erwartete Rendite" an, bis die Kurve des Gesamtvermögens dein gesamtes Leben (bis zum 89. Lebensjahr) abdeckt.

Achte auf: Den blauen Bereich! Er zeigt dir, wie viel deines Vermögens rein durch den Zinseszins entstanden ist – das ist das Geld, das der Markt für dich "erarbeitet" hat.

Tipp für die Aufklärung: Experimentiere mit dem Startalter. Du wirst sehen, dass ein früher Start oft wichtiger ist als eine hohe Sparrate!

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def etf_rechner_optimiert(
    aktuelles_alter,
    renteneintrittsalter,
    rendite_prozent,
    monatliche_einzahlung,
    erhöhung_prozent,
    entnahme_monatlich
):
    # Konstanten & Initialisierung
    lebenserwartung = 89
    sparjahre = max(0, renteneintrittsalter - aktuelles_alter)
    entnahmejahre = max(0, lebenserwartung - renteneintrittsalter)
    
    r_monatlich = rendite_prozent / 100 / 12
    dynamik_jahre = erhöhung_prozent / 100
    steuer_satz = 0.18 # Inkl. Teilfreistellung
    
    kapital = []
    investiert = []
    kontostand = 0
    summe_investiert = 0
    
    # 1. Sparphase
    for monat in range(1, int(sparjahre * 12) + 1):
        # Dynamik greift nach jedem vollen Jahr (12 Monate)
        jahre_vergangen = (monat - 1) // 12
        aktuelle_rate = monatliche_einzahlung * ((1 + dynamik_jahre) ** jahre_vergangen)
        
        kontostand = kontostand * (1 + r_monatlich) + aktuelle_rate
        summe_investiert += aktuelle_rate
        
        kapital.append(kontostand)
        investiert.append(summe_investiert)

    # 2. Entnahmephase
    n_entnahme_monate = int(entnahmejahre * 12)
    kapital_ende_sparphase = kontostand
    invest_ende_sparphase = summe_investiert
    
    for monat in range(1, n_entnahme_monate + 1):
        if kontostand <= 0:
            kapital.append(0)
            investiert.append(invest_ende_sparphase)
            continue
            
        # Steuerberechnung (Näherung via Gewinnanteil)
        anteil_gewinn = max(0, 1 - (invest_ende_sparphase / kontostand))
        steuer = entnahme_monatlich * anteil_gewinn * steuer_satz
        
        kontostand = kontostand * (1 + r_monatlich) - entnahme_monatlich - steuer
        kapital.append(max(0, kontostand))
        investiert.append(invest_ende_sparphase)

    # Datenaufbereitung für Plot
    gesamt_monate = len(kapital)
    zeitachse = np.linspace(aktuelles_alter, aktuelles_alter + gesamt_monate/12, gesamt_monate)
    kapital = np.array(kapital)
    investiert = np.array(investiert)
    zinsanteil_prozent = ((kapital_ende_sparphase - invest_ende_sparphase) / kapital_ende_sparphase * 100) if kapital_ende_sparphase > 0 else 0

    # Plotting
    plt.figure(figsize=(12, 6))
    
    # Flächen füllen für maximale visuelle Wirkung
    plt.fill_between(zeitachse, investiert, color='#2ecc71', alpha=0.3, label="Eingezahltes Kapital")
    plt.fill_between(zeitachse, investiert, kapital, where=(kapital > investiert), color='#3498db', alpha=0.3, label="Zinseszins-Anteil")
    
    plt.plot(zeitachse, kapital, color='#2980b9', linewidth=3, label="Gesamtvermögen")
    plt.plot(zeitachse, investiert, color='#27ae60', linestyle='--', linewidth=2)
    
    # Vertikale Linien
    plt.axvline(renteneintrittsalter, color='#c0392b', linestyle=':', label="Rentenbeginn")
    
    # Styling & Beschriftung
    plt.title(f"Aufklärungs-Tool: Die Macht des Zinseszinses\n"
              f"Ihr Zinsanteil am Sparziel: {zinsanteil_prozent:.1f}% | "
              f"Endvermögen Sparphase: {kapital_ende_sparphase:,.0f} €", fontsize=14)
    plt.ylabel("Kapitalwert (€)")
    plt.xlabel("Alter (Jahre)")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend(loc='upper left')
    
    # Währung an Y-Achse formatieren
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',').replace(',', '.') + ' €'))
    
    plt.tight_layout()
    plt.show()

# Widget-Konfiguration (Zentral gesteuert)
ui_style = {'description_width': '220px'}
ui_layout = {'width': '550px'}

interact(
    etf_rechner_optimiert,
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, step=1, description="Aktuelles Alter", style=ui_style, layout=ui_layout),
    renteneintrittsalter=widgets.IntSlider(value=67, min=25, max=75, step=1, description="Rentenbeginn", style=ui_style, layout=ui_layout),
    rendite_prozent=widgets.FloatSlider(value=7.0, min=0, max=15, step=0.1, description="Erwartete Rendite p.a. (%)", style=ui_style, layout=ui_layout),
    monatliche_einzahlung=widgets.IntSlider(value=200, min=0, max=5000, step=50, description="Start-Sparrate (€)", style=ui_style, layout=ui_layout),
    erhöhung_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Jährliche Dynamik (%)", style=ui_style, layout=ui_layout),
    entnahme_monatlich=widgets.IntSlider(value=2500, min=0, max=20000, step=100, description="Wunsch-Entnahme (€)", style=ui_style, layout=ui_layout)
);

interactive(children=(IntSlider(value=28, description='Aktuelles Alter', layout=Layout(width='550px'), max=70,…